In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

In [2]:

transform = transforms.Compose([
     transforms.ToTensor()
 ])


train_data = datasets.FashionMNIST(root='data', train=True, download=True , transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)


100%|██████████| 26.4M/26.4M [00:00<00:00, 117MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 6.81MB/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4.42M/4.42M [00:00<00:00, 64.1MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 14.8MB/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:
model_with_layers = MLP()
model_with_layers.load_state_dict(torch.load('mlp_model_weights.pth'))
state_dict = torch.load('mlp_model_weights.pth')

<ipython-input-5-8497443de310>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_with_layers.load_state_dict(torch.load('mlp_model_weights.pth'))
<ipython-input-5-84974

In [6]:
def relu(x):
    return torch.max(torch.zeros_like(x), x)


In [7]:
def Mymodel(x):
    x = x.view(-1, 28 * 28)
    x = relu(x @ state_dict['fc1.weight'].T + state_dict['fc1.bias'])
    x = x @ state_dict['fc2.weight'].T + state_dict['fc2.bias']

    return x

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model_with_layers(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy =  100 * correct / total
print(f'Torch model: {accuracy:.2f}%')


Torch model: 88.88%


In [15]:

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = Mymodel(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy =  100 * correct / total
print(f'My model: {accuracy:.2f}%')


My model: 88.88%
